In [1]:
import pandas as pd
from tqdm import tqdm
from catboost import CatBoostClassifier

In [2]:
import pandas as pd

df_vol = pd.read_excel('data\preprocess\df_vol.xlsx')
df_price = pd.read_excel('data\preprocess\df_price.xlsx')
df_req = pd.read_excel('data\preprocess\df_req.xlsx')
df_interest = pd.read_excel('data\preprocess\df_interest.xlsx')
df_mc = pd.read_excel('data\preprocess\df_mc.xlsx')


In [3]:
agg_funcs = {
    'Субъект федерации отп': lambda x: x.mode()[0],  # Мода
    'Субъект федерации наз': lambda x: x.mode()[0],  # Мода
    'Код груза': lambda x: x.mode()[0],  # Мода
    'Гр груза по опер.номен': lambda x: x.mode()[0]  # Мода
}

# Добавляем сумму для всех остальных столбцов
for col in df_price.columns[5:]:  # Пропускаем первые 5 столбцов
    agg_funcs[col] = 'sum'

# Создаем новый DataFrame с агрегированными данными
df_targets = df_price.groupby('ID').agg(agg_funcs).reset_index()

In [4]:
date_cols = ['2022/01', '2022/02', '2022/03', '2022/04',
       '2022/05', '2022/06', '2022/07', '2022/08', '2022/09', '2022/10',
       '2022/11', '2022/12', '2023/01', '2023/02', '2023/03', '2023/04',
       '2023/05', '2023/06', '2023/07', '2023/08', '2023/09', '2023/10',
       '2023/11', '2023/12', '2024/01', '2024/02', '2024/03', '2024/04',
       '2024/05', '2024/06', '2024/07', '2024/08']
last_6_cols = ['2024/03', '2024/04',
       '2024/05', '2024/06', '2024/07', '2024/08']
last_12_cols = ['2023/09', '2023/10',
       '2023/11', '2023/12', '2024/01', '2024/02', '2024/03', '2024/04',
       '2024/05', '2024/06', '2024/07', '2024/08']
last_18_cols = ['2023/03', '2023/04',
       '2023/05', '2023/06', '2023/07', '2023/08', '2023/09', '2023/10',
       '2023/11', '2023/12', '2024/01', '2024/02', '2024/03', '2024/04',
       '2024/05', '2024/06', '2024/07', '2024/08']

cols_train2 = [ '2023/03', '2023/04',
       '2023/05', '2023/06', '2023/07', '2023/08']

cols_for_train1 = ['2023/03', '2023/04','2023/05', '2023/06', '2023/07', '2023/08', '2023/09', '2023/10', '2023/11', '2023/12', '2024/01', '2024/02']
cols_for_train2 = ['2022/03', '2022/04',
       '2022/05', '2022/06', '2022/07', '2022/08', '2022/09', '2022/10',
       '2022/11', '2022/12', '2023/01', '2023/02']

cols_for_now = ['2023/09', '2023/10',
       '2023/11', '2023/12', '2024/01', '2024/02', '2024/03', '2024/04',
       '2024/05', '2024/06', '2024/07', '2024/08']

In [5]:
df_targets['target1'] = (df_targets[last_6_cols].sum(axis=1) == 0).astype(int)
df_targets['target2'] = (df_targets[cols_train2].sum(axis=1) == 0).astype(int)

df_train1 = df_targets[['ID', 'target1']]
df_train2 = df_targets[['ID', 'target2']]

In [6]:
mask = (df_targets[last_6_cols] != 0).any(axis=1)

# Filter df3 based on the mask
df3 = df_targets[mask].copy()[['ID']]

In [7]:
for index, row in tqdm(df3.iterrows()):
    id = row['ID']
    price_info = df_price[df_price.ID == id]
    vol_info = df_vol[df_vol.ID == id]

    # Регион 1
    reg1_mode = price_info['Субъект федерации отп'].mode()
    reg1_count = len(price_info['Субъект федерации отп'].unique())

    # Регион 2
    reg2_mode = price_info['Субъект федерации наз'].mode()
    reg2_count = len(price_info['Субъект федерации наз'].unique())

    # Код груза
    code_mode = price_info['Код груза'].mode()
    code_count = len(price_info['Код груза'].unique())

    # Название груза
    name_mode = price_info['Гр груза по опер.номен'].mode()
    name_count = len(price_info['Гр груза по опер.номен'].unique())

    # Заполнение df3
    df3.loc[index, 'reg1_mode'] = reg1_mode[0] if not reg1_mode.empty else None
    df3.loc[index, 'reg1_count'] = reg1_count
    df3.loc[index, 'reg2_mode'] = reg2_mode[0] if not reg2_mode.empty else None
    df3.loc[index, 'reg2_count'] = reg2_count
    df3.loc[index, 'code_mode'] = code_mode[0] if not code_mode.empty else None
    df3.loc[index, 'code_count'] = code_count
    df3.loc[index, 'name_mode'] = name_mode[0] if not name_mode.empty else None
    df3.loc[index, 'name_count'] = name_count

    # Обработка цен и объемов
    for col in cols_for_now:
        df3.loc[index, f'{col}_sum_price'] = price_info[col].sum()
        df3.loc[index, f'{col}_count_price'] = price_info[col].notnull().sum()

    for col in cols_for_now:
        df3.loc[index, f'{col}_sum_vol'] = vol_info[col].sum()
        df3.loc[index, f'{col}_count_vol'] = vol_info[col].notnull().sum()

    # Обработка запросов
    req = df_req[df_req['ID'] == id]
    req_count = len(req)
    df3.loc[index, 'req_count'] = req_count

    req_mode = req['Тема вопроса'].mode()
    req_count_uniq = len(req['Тема вопроса'].unique())
    df3.loc[index, 'req_mode'] = req_mode[0] if not req_mode.empty else 'Нет'
    df3.loc[index, 'req_count_uniq'] = req_count_uniq

    req_group_mode = req['Группа вопросов'].mode()
    req_group_count_uniq = len(req['Группа вопросов'].unique())
    df3.loc[index, 'req_group_mode'] = req_group_mode[0] if not req_group_mode.empty else 'Нет'
    df3.loc[index, 'req_group_count_uniq'] = req_group_count_uniq

    # Обработка интересов (без ограничения по дате)
    interest = df_interest[df_interest['ID'] == id]
    interest_count = len(interest)
    df3.loc[index, 'interest_count'] = interest_count
    interest_count_uniq_scen = len(interest['Сценарий'].unique())
    df3.loc[index, 'interest_count_uniq_scen'] = interest_count_uniq_scen

    interest_count_uniq_ch = len(interest['Канал первичного интереса'].unique())
    df3.loc[index, 'interest_count_uniq_ch'] = interest_count_uniq_ch

    interest_mode_ch = interest['Канал первичного интереса'].mode()
    df3.loc[index, 'interest_mode_ch'] = interest_mode_ch[0] if not interest_mode_ch.empty else 'Нет'

    interest_mode_scen = interest['Сценарий'].mode()
    df3.loc[index, 'interest_mode_scen'] = interest_mode_scen[0] if not interest_mode_scen.empty else 'Нет'

    interest_mode_state = interest['Состояние'].mode()
    df3.loc[index, 'interest_mode_state'] = interest_mode_state[0] if not interest_mode_state.empty else 'Нет'

    interest_count_bad = len(interest[interest['Состояние'] == 'Завершен неудачно'])
    df3.loc[index, 'interest_count_bad'] = interest_count_bad

    interest_count_good = len(interest[interest['Состояние'] == 'Завершен успешно'])
    df3.loc[index, 'interest_count_good'] = interest_count_good

    # Обработка МСП
    mc = df_mc[df_mc['ID'] == id]
    mc_count = len(mc)
    df3.loc[index, 'mc_count'] = mc_count

    mc_msp_mode = mc['Находится в реестре МСП'].mode()
    df3.loc[index, 'mc_msp_mode'] = mc_msp_mode[0] if not mc_msp_mode.empty else 'unknown'

    mc_size_mode = mc['Размер компании.Наименование'].mode()
    df3.loc[index, 'mc_size_mode'] = mc_size_mode[0] if not mc_size_mode.empty else 'unknown'

    mc_okved_mode = mc['ОКВЭД2.Код'].mode()
    df3.loc[index, 'mc_okved_mode'] = mc_okved_mode[0] if not mc_okved_mode.empty and not mc_okved_mode.isnull().any() else 'unknown'

    # Регион
    mc_region_mode = mc['Регион'].mode()
    df3.loc[index, 'mc_region_mode'] = mc_region_mode[0] if not mc_region_mode.empty else 'unknown'

    # Грузоотправитель
    mc_sender_mode = mc['Грузоотправитель'].mode()
    df3.loc[index, 'mc_sender_mode'] = mc_sender_mode[0] if not mc_sender_mode.empty else 'unknown'

    # Грузополучатель
    mc_receiver_mode = mc['Грузополучатель'].mode()
    df3.loc[index, 'mc_receiver_mode'] = mc_receiver_mode[0] if not mc_receiver_mode.empty else 'unknown'

0it [00:00, ?it/s]

610it [00:11, 54.56it/s]


In [8]:
cat_features = [
    'reg1_mode',
    'reg2_mode',
    'code_mode',
    'name_mode',
    'req_mode',
    'req_group_mode',
    'interest_mode_ch',
    'interest_mode_scen',
    'interest_mode_state',
    'mc_msp_mode',
    'mc_size_mode',
    'mc_okved_mode',
    'mc_region_mode',
    'mc_sender_mode',
    'mc_receiver_mode',
]


In [10]:
model = CatBoostClassifier()
model.load_model('model_catboost.cbm')

In [11]:
# Напишем функцию для инференса модели:
def model_inference(input_data):
    """Функция для инференса модели на новых данных."""
    input_data[list(set(input_data.columns) - set(cat_features))] = input_data[list(set(input_data.columns) - set(cat_features))].astype(float)
    input_data[cat_features] = input_data[cat_features].astype(str)
    predictions = model.predict_proba(input_data.drop(columns='ID'))[:, 1]
    return predictions

In [ ]:
df3[list(set(df3.columns) - set(cat_features))] = df3[list(set(df3.columns) - set(cat_features))].astype(float)
df3[cat_features] = df3[cat_features].astype(str)
pred_real = pd.DataFrame()
pred_real['ID'] = df3['ID']
pred_real['Регион'] = df3['reg1_mode']
pred_real['Вероятность'] = model.predict_proba(df3.drop(columns = 'ID'))[:, 1]
pred_real['ID'] = pred_real['ID'].astype(int).astype(str)

pred_real